In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.metrics import roc_auc_score
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, zscore
from statsmodels.stats.weightstats import ztest
from scipy.stats import chi2_contingency

In [2]:
df = pd.read_csv('/kaggle/input/complete-dataset/complete_dataset.csv')

In [3]:
print(df)

              id  CustomerId         Surname  CreditScore Geography  Gender  \
0            0.0  15674932.0  Okwudilichukwu        668.0    France    Male   
1            1.0  15749177.0   Okwudiliolisa        627.0    France    Male   
2            2.0  15694510.0           Hsueh        678.0    France    Male   
3            3.0  15741417.0             Kao        581.0    France    Male   
4            4.0  15766172.0       Chiemenam        716.0     Spain    Male   
...          ...         ...             ...          ...       ...     ...   
302558   98021.0  15656096.0           Ch'in        516.0    France    Male   
302559  243523.0  15581323.0           White        536.0    France    Male   
302560  193768.0  15589948.0          Disher        579.0     Spain    Male   
302561  119381.0  15626141.0        Mazzanti        753.0    France    Male   
302562  218536.0  15625021.0            Hung        724.0    France  Female   

         Age  Tenure    Balance  NumOfProducts  Has

In [4]:
df = df.drop_duplicates()

In [5]:
def fill_missing_values(df):
    
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[col].fillna(df[col].mean(), inplace=True)
    for col in df.select_dtypes(include=['object']).columns:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    return df
            
fill_missing_values(df)

/tmp/ipykernel_30/1655620123.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/tmp/ipykernel_30/1655620123.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.0,1.567493e+07,Okwudilichukwu,668.000000,France,Male,33.000000,3.000000,0.00000,2.000000,1.000000,0.000000,181449.970000,0.000000
1,1.0,1.574918e+07,Okwudiliolisa,627.000000,France,Male,33.000000,1.000000,0.00000,2.000000,1.000000,1.000000,49503.500000,0.000000
2,2.0,1.569451e+07,Hsueh,678.000000,France,Male,40.000000,10.000000,0.00000,2.000000,1.000000,0.000000,184866.690000,0.000000
3,3.0,1.574142e+07,Kao,581.000000,France,Male,34.000000,2.000000,148882.54000,1.000000,1.000000,1.000000,84560.880000,0.000000
4,4.0,1.576617e+07,Chiemenam,716.000000,Spain,Male,33.000000,5.000000,0.00000,2.000000,1.000000,1.000000,15068.830000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275053,275053.0,1.577413e+07,Cox,575.000000,France,Female,36.000000,4.000000,178032.53000,1.000000,1.000000,1.000000,42181.680000,0.000000
275054,275054.0,1.572846e+07,Ch'iu,712.000000,France,Male,31.000000,2.000000,0.00000,2.000000,1.000000,0.000000,16287.380000,0.000000
275055,275055.0,1.568754e+07,Yegorova,709.000000,France,Female,32.000000,3.000000,0.00000,1.000000,1.000000,1.000000,158816.580000,0.000000
275056,275056.0,1.566394e+07,Tuan,621.000000,France,Female,37.000000,7.000000,87848.39000,1.000000,1.000000,0.000000,24210.560000,0.000000


In [6]:
num_col = df.select_dtypes(include=['number']).columns
print(num_col)

Index(['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')


In [10]:
import numpy as np

def normalize_outliers(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:


        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df[col] = np.where(df[col] < lower_bound, df[col].quantile(0.01), df[col])
        df[col] = np.where(df[col] > upper_bound, df[col].quantile(0.99), df[col])
        
    return df

df = normalize_outliers(df)


In [11]:
from statsmodels.stats.weightstats import ztest

# Extract credit score data for each gender and drop NaNs
male_creditscore = df[df['Gender'] == 'Male']['CreditScore'].dropna()
female_creditscore = df[df['Gender'] == 'Female']['CreditScore'].dropna()


if len(male_creditscore) == 0 or len(female_creditscore) == 0:
    print("One of the groups has no data.")
elif male_creditscore.std() == 0 or female_creditscore.std() == 0:
    print("One of the groups has zero variance.")
else:
    #  Z-test
    z_stat, p_value = ztest(male_creditscore, female_creditscore)

    print(f"Z-test Statistic: {z_stat}")
    print(f"P-value: {p_value}")

    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in CreditScore between males and females.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in CreditScore between males and females.")


Z-test Statistic: 0.3547101689395948
P-value: 0.7228067319871692
Fail to reject the null hypothesis: No significant difference in CreditScore between males and females.


In [13]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
france_bal = df[df['Geography'] == 'France']['Balance'].dropna()
germany_bal = df[df['Geography'] == 'Germany']['Balance'].dropna()

if len(france_bal) == 0 or len(germany_bal) == 0:
    print("One of the groups has no data.")
elif france_bal.std() == 0 or germany_bal.std() == 0:
    print("One of the groups has zero variance.")
else:
    # T-test
    t_stat, p_value = ttest_ind(france_bal, germany_bal, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between France and Germany.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between France and Germany.")


T-test Statistic: -479.60369007082255
P-value: 0.0
Reject the null hypothesis: There is a significant difference in Balance between France and Germany.


In [14]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
france_bal = df[df['Geography'] == 'France']['Balance'].dropna()
spain_bal = df[df['Geography'] == 'Spain']['Balance'].dropna()

# Check for empty groups or zero variance
if len(france_bal) == 0 or len(spain_bal) == 0:
    print("One of the groups has no data.")
elif france_bal.std() == 0 or spain_bal.std() == 0:
    print("One of the groups has zero variance.")
else:
    # Perform T-test
    t_stat, p_value = ttest_ind(france_bal, spain_bal, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between France and Spain.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between France and Spain.")


T-test Statistic: -8.287091861047331
P-value: 1.173727696570248e-16
Reject the null hypothesis: There is a significant difference in Balance between France and Spain.


In [18]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
germany_bal = df[df['Geography'] == 'Germany']['Balance'].dropna()
spain_bal = df[df['Geography'] == 'Spain']['Balance'].dropna()

# Check for empty groups, zero variance, and data types
if len(germany_bal) == 0 or len(spain_bal) == 0:
    print("One of the groups has no data.")
elif germany_bal.std() == 0 or spain_bal.std() == 0:
    print("One of the groups has zero variance.")
elif not pd.api.types.is_numeric_dtype(germany_bal) or not pd.api.types.is_numeric_dtype(spain_bal):
    print("One of the groups contains non-numeric data.")
else:
    # T-test
    t_stat, p_value = ttest_ind(germany_bal, spain_bal, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between Germany and Spain.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between Germany and Spain.")


T-test Statistic: 314.7965768705784
P-value: 0.0
Reject the null hypothesis: There is a significant difference in Balance between Germany and Spain.


In [19]:
contingency_table = pd.crosstab(df['Gender'], df['Geography'])
print(contingency_table)

Geography  France  Germany  Spain
Gender                           
Female      66936    27271  25758
Male        90451    30274  34368


In [20]:
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant association between Gender and Geography.")
else:
    print("Fail to reject the null hypothesis: No significant association between Gender and Geography.")

Chi-squared Statistic: 423.6751702165623
P-value: 1.0002434768067043e-92
Degrees of Freedom: 2
Expected Frequencies:
[[68643.45503494 25097.92816424 26223.61680082]
 [88743.54496506 32447.07183576 33902.38319918]]
Reject the null hypothesis: There is a significant association between Gender and Geography.
